In [ ]:
from glob import glob
import pandas as pd
DATA_DIR = "/xxx/part-003756-a894b46e.jsonl"
from tqdm import tqdm
import json
import re
import random

In [ ]:
def format_example(item, cot=False):
    example = item["q_main"] + "\n"
    choice_map = {"A":"option_a", "B":"option_b", "C":"option_c", "D":"option_d"} 
    choice_list = []
    for choice in ["A","B","C","D"]:
        if item[choice_map[choice]] != "":
            example += f'{choice}. {item[choice_map[choice]]}\n'
            choice_list.append(item[choice_map[choice]])
    answer = item["std_ans"]
    if len(choice_list) != 4:
        choice_list = None
    return example, answer, choice_list

all_data = []
data_source = "chinese_choice"
idx = 0
instruction = "请一步步思考以下选择题的每个选项，然后再给出最终答案。\n\n 选择题：\n"
with open(DATA_DIR, 'r') as file:
    for _, line in enumerate(file):
        # 解码JSON对象
        json_obj = json.loads(line)
        if json_obj["std_ans"]=="":
            continue
        # 200w数据这块continue
        example, answer, choice_list = format_example(json_obj, False)
        if choice_list is None:
            continue
        data = {
        "data_source": data_source,
        "prompt": [{
            "role": "user",
            "content": instruction + example
        }],
        "ability": "math",
        "reward_model": {
            "style": "rule",
            "ground_truth": answer
        },
        "extra_info": {
            # 后边处理
            'split': None,
            'index': idx,
            'choice_list': choice_list,
        }
        }
        idx += 1
        all_data.append(data)


In [ ]:
# 随机打乱
random.seed(42)
random.shuffle(all_data)
train_data_num = 8000
test_data_num = 1000
test_data_50_num = 50
train_data = all_data[:train_data_num]
test_data = all_data[train_data_num:train_data_num + test_data_num]
test_data_50 = all_data[train_data_num + test_data_num:train_data_num + test_data_num + test_data_50_num]
for d in train_data:
    d["extra_info"]["split"] = "train"
for d in test_data:
    d["extra_info"]["split"] = "test"
# 保存成parquet格式
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)
test_50_df = pd.DataFrame(test_data_50)
print(train_df.iloc[0])
print(len(train_df))
train_df.to_parquet("/xxx/wanjuan/wanjuan_exam_train.parquet")
test_df.to_parquet("/xxx/wanjuan/wanjuan_exam_test.parquet")
test_50_df.to_parquet("/xxx/wanjuan/wanjuan_exam_test_50.parquet")

data_source                                        chinese_choice
prompt          [{'role': 'user', 'content': '请一步步思考以下选择题的每个选项...
ability                                                      math
reward_model               {'style': 'rule', 'ground_truth': 'A'}
extra_info      {'split': 'train', 'index': 73431, 'choice_lis...
Name: 0, dtype: object
